In [ ]:
#INPUTS
percentEVSales = float(input("What is the percent increase of EV sales in 2035? (eg. 5 for 5%) "))
numberOfChargers = int(input("What number of chargers is being installed? "))
#Average kilometers driven per year in BC is 13,100kms
kmDriven = 13100
evSales = 18 #Percent, From https://dailyhive.com/vancouver/bc-electric-vehicle-sales-uptake-2022-statistics

What is the percent increase of EV sales in 2035? (eg. 5 for 5%) 7.22
What number of chargers is being installed? 10


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import math

In [ ]:
Vehicles = pd.read_csv('/content/drive/MyDrive/CHE571 Group 20/Copy of Vehicle Population - 2022 Passenger Vehicles_Full _data.csv')
EmissionsFactors = pd.read_csv('/content/drive/MyDrive/CHE571 Group 20/EmissionsFactorsTechscape.csv')
SizeMap = pd.read_csv('/content/drive/MyDrive/CHE571 Group 20/SizeMap.csv')
vehicleSales2022 = pd.read_csv('/content/drive/MyDrive/CHE571 Group 20/ev_sales_2022.csv')

In [ ]:
# Q4 2022 car sales in BC
carSales = int(vehicleSales2022.query("Month=='Total'")["Unit Sales"])
bcPopulationPercent = 662248 / 39498018  # Vancouver pop / canada pop
#Removed BC population 5403528 replaced with Vancouver
vanCarSales = math.ceil(carSales * bcPopulationPercent)
vanCarSales

24927

In [ ]:
#Need to calculate baseline % of vehicles that are EV/gas
#Then use study values to get % increase/decrease for 2035
isEV_counts = Vehicles['Electric Vehicle Included'].value_counts()
percentReplaced = vanCarSales/sum(isEV_counts)
percentageEV = (isEV_counts['Yes'] / len(Vehicles)) * 100
percentageICE = 100 - percentageEV
salesIncreasePerYear = (100-18)/(2035-2023) #Assuming increase in EV purchases is linear and 100% of new purchases are EVs by 2035
percentageEV2035 = percentageEV
for i in range(2035-2023):
  if percentageEV2035 < 100:
    percentageEV2035 = percentReplaced * (evSales + i*salesIncreasePerYear) + (1-percentReplaced) * percentageEV2035
  else:
    pass
percentageICE2035 = 100 - percentageEV2035
ICEadjustmentFactor2035 = percentageICE2035/percentageICE
EVadjustmentFactor2035 = percentageEV2035/percentageEV
percentageEV2035

79.64484294870884

#Gas vehicles baseline

In [ ]:
VehiclesMapped = Vehicles.merge(SizeMap, on='Body Style', how='left')
GasVehiclesEmissions = VehiclesMapped.merge(EmissionsFactors[EmissionsFactors['Powertrain type'] == 'Gasoline'], on='Size', how='left')
GasVehiclesEmissions = GasVehiclesEmissions[GasVehiclesEmissions['Electric Vehicle Included'] == 'No']
GasBaselineEmissions_WTP = GasVehiclesEmissions.groupby('Size')['Well to pump emissions (g/km)'].sum().reset_index()
GasBaselineEmissions_PTW = GasVehiclesEmissions.groupby('Size')['Pump to wheel emissions (g/km)'].sum().reset_index()

In [ ]:
GasBaselineEmissions_WTP

,Size,Well to pump emissions (g/km)
0,Compact,3.722830e+05
1,High midsize,9.420564e+05
2,Midsize,5.069035e+05
3,Pickup,1.958070e+04
4,Small SUV,1.505609e+06


In [ ]:
GasBaselineEmissions_PTW

,Size,Pump to wheel emissions (g/km)
0,Compact,1.555670e+06
1,High midsize,3.939465e+06
2,Midsize,2.128053e+06
3,Pickup,8.197333e+04
4,Small SUV,6.272717e+06


In [ ]:
GasBaselineEmissions_WTP2035 = GasBaselineEmissions_WTP['Well to pump emissions (g/km)'] * ICEadjustmentFactor2035
GasBaselineEmissions_PTW2035 = GasBaselineEmissions_PTW['Pump to wheel emissions (g/km)'] * ICEadjustmentFactor2035

#EVs Baseline

In [ ]:
EVBaselineEmissions = VehiclesMapped.merge(EmissionsFactors[EmissionsFactors['Powertrain type'] == 'EV'], on='Size', how='left')
EVBaselineEmissions = EVBaselineEmissions[EVBaselineEmissions['Electric Vehicle Included'] == 'Yes']
EVBaselineEmissions_WTP = EVBaselineEmissions.groupby('Size')['Well to pump emissions (g/km)'].sum().reset_index()
EVBaselineEmissions_PTW = EVBaselineEmissions.groupby('Size')['Pump to wheel emissions (g/km)'].sum().reset_index()

In [ ]:
EVBaselineEmissions_WTP2035 = EVBaselineEmissions_WTP['Well to pump emissions (g/km)'] * EVadjustmentFactor2035
EVBaselineEmissions_PTW2035 = EVBaselineEmissions_PTW['Pump to wheel emissions (g/km)'] * EVadjustmentFactor2035

#Baseline in one dataframe (g/km driven)

In [ ]:
BaselineEmissions = pd.concat([GasBaselineEmissions_WTP,
                               GasBaselineEmissions_PTW['Pump to wheel emissions (g/km)'],
                               GasBaselineEmissions_WTP2035, GasBaselineEmissions_PTW2035,
                               EVBaselineEmissions_WTP['Well to pump emissions (g/km)'], EVBaselineEmissions_PTW['Pump to wheel emissions (g/km)'],
                               EVBaselineEmissions_WTP2035, EVBaselineEmissions_PTW2035], axis = 1)
BaselineEmissions.columns = ['Size', 'Gas WTP', 'Gas PTW', 'Gas WTP 2035', 'Gas PTW 2035', 'EV WTP', 'EV PTW', 'EV WTP 2035', 'EV PTW 2035']
BaselineEmissions

,Size,Gas WTP,Gas PTW,Gas WTP 2035,Gas PTW 2035,EV WTP,EV PTW,EV WTP 2035,EV PTW 2035
0,Compact,3.722830e+05,1.555670e+06,78159.262872,3.266065e+05,2140.629078,0.0,55978.073488,0.0
1,High midsize,9.420564e+05,3.939465e+06,197780.779958,8.270742e+05,32233.089340,0.0,842904.668849,0.0
2,Midsize,5.069035e+05,2.128053e+06,106422.251382,4.467758e+05,23341.866855,0.0,610396.612767,0.0
3,Pickup,1.958070e+04,8.197333e+04,4110.885269,1.720996e+04,55.923546,0.0,1462.417014,0.0
4,Small SUV,1.505609e+06,6.272717e+06,316096.185567,1.316931e+06,33971.690262,0.0,888369.589049,0.0


#Project Emissions

In [ ]:
salesIncreasePerYearProject = (100-18+percentEVSales)/(2035-2023)
percentageEV2035Project = percentageEV
for i in range(2035-2023):
  if percentageEV2035Project < 100:
    percentageEV2035Project = percentReplaced * (evSales + i*salesIncreasePerYearProject) + (1-percentReplaced) * percentageEV2035Project
  else:
    pass
percentageICE2035Project = 100 - percentageEV2035Project
ICEadjustmentFactor2035Project = percentageICE2035Project/percentageICE
EVadjustmentFactor2035Project = percentageEV2035Project/percentageEV
percentageEV2035Project

85.0824378836156

#Gas vehicles project

In [ ]:
GasProjectEmissions_WTP2035 = GasBaselineEmissions_WTP['Well to pump emissions (g/km)'] * ICEadjustmentFactor2035Project
GasProjectEmissions_PTW2035 = GasBaselineEmissions_PTW['Pump to wheel emissions (g/km)'] * ICEadjustmentFactor2035Project

#EVs project

In [ ]:
EVProjectEmissions_WTP2035 = EVBaselineEmissions_WTP['Well to pump emissions (g/km)'] * EVadjustmentFactor2035Project
EVProjectEmissions_PTW2035 = EVBaselineEmissions_PTW['Pump to wheel emissions (g/km)'] * EVadjustmentFactor2035Project

#Charger lifecycle project

In [ ]:
chargerLifecycle = 4737.6 + 71.0 + 86.3 + 3310.7 #Production, transportation, installation, and recycling in kg CO2
totalChargerEmissions = numberOfChargers * chargerLifecycle #in kg CO2
totalChargerEmissions

82056.0

#TOTAL

In [ ]:
totalBaseline = (sum(EVBaselineEmissions_WTP2035) + sum(EVBaselineEmissions_WTP2035) + sum(GasBaselineEmissions_WTP2035) + sum(GasBaselineEmissions_PTW2035))*13100/1000
totalProject = (sum(EVProjectEmissions_WTP2035) + sum(EVProjectEmissions_WTP2035) + sum(GasProjectEmissions_WTP2035) + sum(GasProjectEmissions_PTW2035))*13100/1000
netCarbon = totalProject + totalChargerEmissions - totalBaseline
AllEmissions = pd.concat([BaselineEmissions
                         ,GasProjectEmissions_WTP2035
                         ,GasProjectEmissions_PTW2035
                         ,EVProjectEmissions_WTP2035
                         ,EVProjectEmissions_PTW2035], axis = 1)
AllEmissions.columns = ['Size', 'Gas WTP', 'Gas PTW', 'Gas WTP 2035 Baseline', 'Gas PTW 2035 Baseline', 'EV WTP', 'EV PTW', 'EV WTP 2035 Baseline', 'EV PTW 2035 Baseline', 'Gas WTP 2035 Project', 'Gas PTW 2035 Project', 'EV WTP 2035 Project', 'EV PTW 2035 Project']
print("The net CO2 reduction in 2035 is "+str(netCarbon)+" kg CO2e")
AllEmissions

The net CO2 reduction in 2035 is -8354721.353542596 kg CO2e


,Size,Gas WTP,Gas PTW,Gas WTP 2035 Baseline,Gas PTW 2035 Baseline,EV WTP,EV PTW,EV WTP 2035 Baseline,EV PTW 2035 Baseline,Gas WTP 2035 Project,Gas PTW 2035 Project,EV WTP 2035 Project,EV PTW 2035 Project
0,Compact,3.722830e+05,1.555670e+06,78159.262872,3.266065e+05,2140.629078,0.0,55978.073488,0.0,57280.111174,239358.128124,59799.866307,0.0
1,High midsize,9.420564e+05,3.939465e+06,197780.779958,8.270742e+05,32233.089340,0.0,842904.668849,0.0,144946.416430,606132.887747,900452.326528,0.0
2,Midsize,5.069035e+05,2.128053e+06,106422.251382,4.467758e+05,23341.866855,0.0,610396.612767,0.0,77993.038401,327425.917001,652070.240425,0.0
3,Pickup,1.958070e+04,8.197333e+04,4110.885269,1.720996e+04,55.923546,0.0,1462.417014,0.0,3012.719883,12612.557899,1562.260658,0.0
4,Small SUV,1.505609e+06,6.272717e+06,316096.185567,1.316931e+06,33971.690262,0.0,888369.589049,0.0,231655.519585,965131.047787,949021.274694,0.0
